# Deelstudie 1: Retrospectieve analyse van historische YouTube-data - Invloed van comment-sentiment op het aantal views

**Hypothese:** Video’s waarvan de comments een positief sentiment hebben, vertonen een significant hoger aantal views

**Manier van werken:**
- Beschrijvende statistiek
- Normaliteitstoets
- Correlatieanalyse
- Categorie-analyse: views tussen positief/negatief/neutraal sentiment
- Post-hoc analyse indien significant verschil

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
# General settings
sns.set_theme(palette='muted')

In [ ]:
# Import video data
videos = pd.read_excel('../data/videos.xlsx', index_col=0)
videos

In [ ]:
# Import comments data
comments = pd.read_excel('../data/comments.xlsx', index_col=1)
comments

In [ ]:
# Get info about the video data
print(videos.shape)
print(videos.dtypes)

In [ ]:
# Get info about the comments data
print(comments.shape)
print(comments.dtypes)

## STAP 1: Sentiment berekenen

### 1.1 Sentimentscore per comment

In [ ]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Calculate sentiment scores for Deepl translated comments
comments['sentiment_deepl'] = comments['en_deepl'].apply(
    lambda x: analyzer.polarity_scores(x)['compound'] if isinstance(x, str) else np.nan
)

In [ ]:
# Check if Google translation exists and calculate; otherwise use Deepl score
comments['sentiment_google'] = comments.apply(
    lambda row: analyzer.polarity_scores(row['en_google'])['compound']
    if isinstance(row['en_google'], str)
    else row['sentiment_deepl'],
    axis=1
)

In [ ]:
# Calculate the sentiment score as the average of the two methods
comments['sentiment'] = comments[['sentiment_google', 'sentiment_deepl']].mean(axis=1, skipna=True)

# Filter out comments without a valid sentiment score
comments = comments[comments['sentiment'].notna()].copy()

### 1.2 Sentimentscore per video

In [ ]:
# Calculate the average comment sentiment per video
video_sentiment_score = comments.groupby(comments.index)['sentiment'].mean()

# Add data to videos dataframe
videos['sentiment_score'] = videos.index.to_series().map(video_sentiment_score)

# Filter videos to only include those with comments
# Alternative: regard comments without comments as neutral --> this introduces a bias, so I did not opt for this
videos = videos[videos['sentiment_score'].notna()].copy()

In [ ]:
# Categorize sentiment score
def categorize_sentiment(score):
    if pd.isna(score):
        return 'geen data' # There shouldn't be any NaN values, but just in case
    elif score > 0.05:
        return 'positief'
    elif score < -0.05:
        return 'negatief'
    else:
        return 'neutraal'

comments['sentiment_category'] = comments['sentiment'].apply(categorize_sentiment)
videos['sentiment_category'] = videos['sentiment_score'].apply(categorize_sentiment)

## STAP 2: Beschrijvende statistiek

### 2.1 Comment niveau

In [ ]:
# Get the number of comments per sentiment category
comment_counts = comments['sentiment_category'].value_counts(dropna=False).to_frame(name='Aantal comments')
comment_counts['Percentage (%)'] = (comment_counts['Aantal comments'] / comment_counts['Aantal comments'].sum() * 100).round(3)
comment_counts

In [ ]:
# Sentiment score statistics
comments['sentiment'].describe().round(3)

In [ ]:
# Calc shares per sentiment category
comment_counts = comments['sentiment_category'].value_counts()
percentages = comment_counts / comment_counts.sum() * 100

# Plot pie chart
category_colors = ['green', 'red', 'lightblue']
plt.figure(figsize=(6, 6))
wedges, texts, autotexts = plt.pie(
    percentages,
    autopct='%.1f%%',
    startangle=90,
    colors=category_colors,
)
plt.legend(wedges, percentages.index, title='Sentimentcategorie', loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Verdeling van comments per sentimentcategorie')
plt.tight_layout()
plt.show()

In [ ]:
# Create side-by-side plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
sns.histplot(comments['sentiment'].dropna(), kde=True, ax=axes[0])
axes[0].set_title('Distributie sentimentscore (per comment)')
axes[0].set_xlabel('Sentimentscore')
axes[0].set_ylabel('Frequentie')

# Boxplot
sns.boxplot(x=comments['sentiment'].dropna(), ax=axes[1])
axes[1].set_title('Boxplot sentimentscore (per comment)')
axes[1].set_xlabel('Sentimentscore')

plt.tight_layout()
plt.show()

In [ ]:
# Set sentiment categories
categories = ['positief', 'neutraal', 'negatief']
n_cat = len(categories)

# Subplot settings
fig, axes = plt.subplots(2, n_cat, figsize=(5 * n_cat, 8))

# Plot per category
for i, cat in enumerate(categories):
    subset = comments[comments['sentiment_category'] == cat]['sentiment'].dropna()

    # Calc statistics
    mean = subset.mean()
    median = subset.median()
    std = subset.std()

    # Histogram
    sns.histplot(subset, kde=True, ax=axes[0, i])
    axes[0, i].set_title(f'Distributie - {cat}')
    axes[0, i].set_xlabel('Sentimentscore')
    axes[0, i].set_ylabel('Frequentie')

    # Annotations
    axes[0, i].axvline(mean, color='red', linestyle='--', label=f'Gemiddelde = {mean:.2f}')
    axes[0, i].axvline(median, color='green', linestyle='--', label=f'Mediaan = {median:.2f}')
    axes[0, i].legend()

    # Boxplot
    sns.boxplot(x=subset, ax=axes[1, i])
    axes[1, i].set_title(f'Boxplot - {cat}')
    axes[1, i].set_xlabel('Sentimentscore')
    axes[1, i].set_yticks([])

# Render plots
plt.tight_layout()
plt.show()

### 2.2 Video niveau

In [ ]:
# Get the number of comments per sentiment category per video
video_counts = videos['sentiment_category'].value_counts(dropna=False).to_frame(name='Aantal video\'s')
video_counts['Percentage (%)'] = (video_counts['Aantal video\'s'] / video_counts['Aantal video\'s'].sum() * 100).round(3)
video_counts

In [ ]:
# Sentiment score statistics per video
videos['sentiment_score'].describe().round(3)

In [ ]:
# Share videos per sentiment category
video_counts = videos['sentiment_category'].value_counts()
video_percentages = video_counts / video_counts.sum() * 100

# Plot pie chart
category_colors = ['green', 'red', 'lightblue']
plt.figure(figsize=(6, 6))
wedges, texts, autotexts = plt.pie(
    video_percentages,
    autopct='%.1f%%',
    startangle=90,
    colors=category_colors
)
plt.legend(wedges, video_percentages.index, title='Sentimentcategorie', loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('Verdeling van video\'s per sentimentcategorie')
plt.tight_layout()
plt.show()

In [ ]:
# Create side-by-side plots for average sentiment per video
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram
sns.histplot(videos['sentiment_score'].dropna(), kde=True, ax=axes[0])
axes[0].set_title('Distributie van gemiddelde sentimentscore (per video)')
axes[0].set_xlabel('Gemiddelde sentimentscore')
axes[0].set_ylabel('Frequentie')

# Boxplot
sns.boxplot(x=videos['sentiment_score'].dropna(), ax=axes[1])
axes[1].set_title('Boxplot van gemiddelde sentimentscore (per video)')
axes[1].set_xlabel('Gemiddelde sentimentscore')

plt.tight_layout()
plt.show()

In [ ]:
# Define sentiment categories to compare
categories = ['positief', 'neutraal', 'negatief']
n_cat = len(categories)

# Create a 2-row layout: histogram on top, boxplot below
fig, axes = plt.subplots(2, n_cat, figsize=(5 * n_cat, 8))

# Loop over each category to create individual plots
for i, cat in enumerate(categories):
    subset = comments[comments['sentiment_category'] == cat]['sentiment'].dropna()

    # Calculate statistics
    mean = subset.mean()
    median = subset.median()
    std = subset.std()

    # Plot histogram with KDE
    sns.histplot(subset, kde=True, ax=axes[0, i])
    axes[0, i].set_title(f'Distributie - {cat}')
    axes[0, i].set_xlabel('Sentimentscore')
    axes[0, i].set_ylabel('Frequentie')

    # Add lines for mean and median
    axes[0, i].axvline(mean, color='red', linestyle='--', label=f'Gemiddelde = {mean:.2f}')
    axes[0, i].axvline(median, color='green', linestyle='--', label=f'Mediaan = {median:.2f}')
    axes[0, i].legend()

    # Plot boxplot
    sns.boxplot(x=subset, ax=axes[1, i])
    axes[1, i].set_title(f'Boxplot - {cat}')
    axes[1, i].set_xlabel('Sentimentscore')
    axes[1, i].set_yticks([])

# Final layout adjustments
plt.tight_layout()
plt.show()

In [ ]:
# Visualize distribution of views per sentiment category
sns.boxplot(data=videos, x='sentiment_category', y='views')
plt.title('Boxplot van views per sentimentcategorie')
plt.xlabel('Sentimentcategorie')
plt.ylabel('Aantal views')
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.tight_layout()
plt.show()

## STAP 3: Normaliteitstoets

In [ ]:
# Normality test with Shapiro-Wilk test

# Views
shapiro_views = stats.shapiro(videos['views'])
print("Shapiro-Wilk test voor views:")
print(f"  W-statistic: {shapiro_views.statistic:.4f}")
print(f"  p-value: {shapiro_views.pvalue:.4e}")
skew_views = videos['views'].skew()
print(f"Skewness van views: {skew_views:.4f}")

# Sentiment score
shapiro_sentiment = stats.shapiro(videos['sentiment_score'].dropna())
print("Shapiro-Wilk test voor gemiddelde sentimentscore:")
print(f"  W-statistic: {shapiro_sentiment.statistic:.4f}")
print(f"  p-value: {shapiro_sentiment.pvalue:.4e}")
skew_sentiment = videos['sentiment_score'].dropna().skew()
print(f"Skewness van gemiddelde sentimentscore: {skew_sentiment:.4f}")

### Resultaten en interpretatie normaliteitstoets

De Shapiro-Wilk test en skewnessanalyse leverden de volgende resultaten op:

- **Aantal views per video**
  - W-statistic: 0.3260
  - p-waarde: 2.1602e-37
  - Skewness: 8.0701

- **Gemiddelde sentimentscore per video**
  - W-statistic: 0.9657
  - p-waarde: 1.0692e-08
  - Skewness: -0.3477

**Interpretatie**

De verdeling van het aantal views per video **wijkt sterk af van normaliteit**. Dit blijkt uit een extreem lage p-waarde (p < 0.001) en een hoge positieve skewness (> 8), wat wijst op een uitgesproken rechts-scheve verdeling. Een log-transformatie van deze variabele is daarom aangewezen alvorens verder te gaan met correlatie- of regressieanalyse `(Field, 2018)`.

De verdeling van de gemiddelde sentimentscore per video is eveneens significant verschillend van normaal (p < 0.001), hoewel de skewnesswaarde van -0.35 binnen een aanvaardbare marge ligt (|skew| < 1). De afwijking is dus mild en suggereert een redelijk symmetrische verdeling. Visuele controle via histogram en boxplot blijft aanbevolen om deze statistische indicatie te bevestigen.

Op basis van deze resultaten wordt aanbevolen om in verdere analyses zowel **Pearson** (voor lineaire relaties) als **Spearman** (voor monotone relaties) te rapporteren.

In [ ]:
# Log-transformation
videos['views_log'] = np.log1p(videos['views'])
videos['sentiment_score_log'] = np.log1p(videos['sentiment_score'])

# Redo Shapiro-Wilk test with log-transformed data
shapiro_views_log = stats.shapiro(videos['views_log'])
shapiro_sentiment_score_log = stats.shapiro(videos['sentiment_score_log'])

# Views log
print(f'\nShapiro-Wilk test log-views:')
print(f'  W-statistic: {shapiro_views_log.statistic:.4f}')
print(f'  p-value: {shapiro_views_log.pvalue:.4e}')
print(f'Skewness log-views: {videos["views_log"].skew():.4f}')

# Sentiment score log
print(f'\nShapiro-Wilk test log-sentiment-score:')
print(f'  W-statistic: {shapiro_sentiment_score_log.statistic:.4f}')
print(f'  p-value: {shapiro_sentiment_score_log.pvalue:.4e}')
print(f'Skewness sentiment_score_log: {videos["sentiment_score_log"].skew():.4f}')

In [ ]:
# Create side-by-side plots for log-transformed views and sentiment score
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Histogram log-views
sns.histplot(videos['views_log'].dropna(), kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distributie van log-getransformeerde views')
axes[0, 0].set_xlabel('Log(views)')
axes[0, 0].set_ylabel('Frequentie')

# Histogram log-sentiment
sns.histplot(videos['sentiment_score_log'].dropna(), kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Distributie van log-getransformeerde sentimentscore')
axes[0, 1].set_xlabel('Log(sentimentscore)')
axes[0, 1].set_ylabel('Frequentie')

# Boxplot log-views
sns.boxplot(x=videos['views_log'].dropna(), ax=axes[1, 0])
axes[1, 0].set_title('Boxplot van log-getransformeerde views')
axes[1, 0].set_xlabel('Log(views)')
axes[1, 0].set_yticks([])

# Boxplot log-sentiment
sns.boxplot(x=videos['sentiment_score_log'].dropna(), ax=axes[1, 1])
axes[1, 1].set_title('Boxplot van log-getransformeerde sentimentscore')
axes[1, 1].set_xlabel('Log(sentimentscore)')
axes[1, 1].set_yticks([])

plt.tight_layout()
plt.show()

### Resultaten en interpretatie log-transformatie

Na het toepassen van een log-transformatie op de variabelen `views` en `sentiment_score` werd de normaliteit opnieuw geëvalueerd met behulp van de Shapiro-Wilk test en de skewness:

- **Log-views**
  - W-statistic: 0.9881
  - p-waarde: 1.0422e-03
  - Skewness: 0.4051

- **Log-sentiment-score**
  - W-statistic: 0.6295
  - p-waarde: 7.0732e-30
  - Skewness: -6.1065

**Interpretatie**

De log-transformatie van `views` heeft de verdeling duidelijk verbeterd. De skewness is teruggebracht tot een aanvaardbaar niveau (|skew| < 1), wat wijst op een redelijk symmetrische verdeling. Ondanks de nog steeds significante p-waarde (p < 0.05), suggereert de skewnesswaarde dat deze variabele nu voldoende benaderd wordt door een normale verdeling. Verdere analyses kunnen dus met `views_log` worden uitgevoerd, inclusief Pearson-correlatie of lineaire regressie.

De log-transformatie van `sentiment_score` daarentegen is **niet succesvol gebleken**. De skewness blijft extreem negatief, en de W-statistic en p-waarde wijzen op een ernstige afwijking van normaliteit. Deze verdeling wordt dus niet genormaliseerd door een log-transformatie. Daarom is het **niet aangewezen om met `sentiment_score_log` verder te werken**. In plaats daarvan wordt geadviseerd om de oorspronkelijke (`niet-getransformeerde`) score te behouden en gebruik te maken van niet-parametrische methoden zoals de **Spearman-correlatie** of Kruskal-Wallis-toets.

## STAP 4: Correlatie-analyse

In [ ]:
# Spearman correlation
spearman_corr, spearman_p = stats.spearmanr(videos['sentiment_score'], videos['views_log'])
print(f"Spearman-correlatie:\n  ρ = {spearman_corr:.4f}, p = {spearman_p:.4e}")

# Pearson-correlation
pearson_corr, pearson_p = stats.pearsonr(videos['sentiment_score'], videos['views_log'])
print(f"Pearson-correlatie:\n  r = {pearson_corr:.4f}, p = {pearson_p:.4e}")

In [ ]:
# Scatterplot
plt.figure(figsize=(8, 6))
sns.regplot(x='sentiment_score', y='views_log', data=videos, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red'})
plt.title('Relatie tussen sentimentscore en log(views)')
plt.xlabel('Gemiddelde sentimentscore (per video)')
plt.ylabel('Log(aantal views)')
plt.tight_layout()
plt.show()

### Resultaten en interpretatie correlatieanalyse

**Spearman-correlatie:**
- Coëfficiënt: -0.0102
- Betekenis: Er is **geen monotone relatie** tussen de gemiddelde sentimentscore van comments en het (log-getransformeerde) aantal views. De correlatiecoëfficiënt ligt dicht bij nul en is niet statistisch significant (*p* = 0.8304).

**Pearson-correlatie:**
- Coëfficiënt: 0.0090
- Betekenis: Ook de lineaire samenhang tussen sentiment en views is verwaarloosbaar en niet significant (*p* = 0.8489). Deze bevinding bevestigt dat video's met meer positief sentiment in de comments **niet noodzakelijk meer views genereren**.

**Interpretatie**

Beide correlatiecoëfficiënten wijzen in dezelfde richting: er is **geen statistisch significante relatie** tussen het sentiment van comments en het aantal views van een video. De Spearman-coëfficiënt toont aan dat ook een monotone samenhang ontbreekt, wat betekent dat video’s met meer positief (of negatief) sentiment in de comments niet systematisch hogere (of lagere) kijkcijfers vertonen.

Het scatterplot met regressielijn bevestigt deze observatie visueel. De puntenwolk vertoont geen duidelijke trend, en de regressielijn is nagenoeg horizontaal. De spreiding is groot en er zijn geen aanwijzingen voor structurele verbanden.

Deze bevindingen spreken de oorspronkelijke hypothese tegen, waarin werd verondersteld dat een positief comment-sentiment bijdraagt aan het vergroten van het bereik. Een mogelijke verklaring is dat sentimentscores van comments te ver verwijderd staan van de factoren die kijkers aantrekken, zoals inhoud, thumbnail, onderwerp of distributiemoment. Alternatief kan het sentiment eerder een reflectie zijn van het publiek *na* het bekijken van de video, en dus niet causaal bijdragen aan de verspreiding ervan.

## STAP 5: Categorie-analyse

In [ ]:
# Compare distribution of views per sentiment category
# (Boxplots were already created above)

# Check normality of views per group
for cat in categories:
    subset = videos[videos['sentiment_category'] == cat]['views']
    stat, p = stats.shapiro(subset)
    print(f'Shapiro-Wilk test voor {cat} - W = {stat:.4f}, p = {p:.4f}')

# Since normality is violated, we use Kruskal-Wallis test
kruskal_result = stats.kruskal(
    *[videos[videos['sentiment_category'] == cat]['views'] for cat in categories]
)
print("\nKruskal-Wallis testresultaat:")
print(f"  H-statistic: {kruskal_result.statistic:.4f}")
print(f"  p-value: {kruskal_result.pvalue:.4e}")

### Resultaten en interpretatie categorie-analyse

Om te onderzoeken of het aantal views verschilt tussen video's met een positief, neutraal of negatief gemiddeld commentsentiment, werd een Kruskal-Wallis-toets uitgevoerd. Aangezien de normaliteit van het aantal views binnen alle drie de sentimentcategorieën werd verworpen (Shapiro-Wilk p < 0.001), werd deze niet-parametrische toets als geschikt beschouwd.

**Kruskal-Wallis testresultaten:**
- H-statistic: 15.9776
- p-waarde: 0.0003

Deze resultaten wijzen op een **statistisch significant verschil** in het aantal views tussen minstens twee sentimentcategorieën (*p* < 0.001). Hoewel de correlatieanalyse eerder geen significante samenhang vond tussen sentimentscore en views (zie stap 4), suggereert deze analyse dat het gemiddelde bereik van video's **wel verschilt** op basis van het *type sentiment* in de comments.

Om te bepalen **welke groepen significant van elkaar verschillen**, werd een post-hocanalyse met de Dunn’s test en Bonferroni-correctie uitgevoerd (zie stap 6).

**Interpretatie**

De significantie van de Kruskal-Wallis test bevestigt dat sentimentscore in categorische vorm — positief, neutraal, negatief — **wél informatief** is voor het verklaren van verschillen in bereik. Dit contrasteert met de afwezigheid van correlatie bij de continue variabele, wat doet vermoeden dat het effect van sentiment op bereik eerder **drempelgebaseerd of niet-lineair** is. Positief sentiment zou bijvoorbeeld pas effect sorteren wanneer het duidelijk overheerst, terwijl kleine schommelingen in sentiment onvoldoende impact hebben.

Deze bevindingen nuanceren de initiële hypothese (H3): hoewel er geen geleidelijke stijging is van views bij oplopend sentiment, lijken video’s met overwegend positief commentaar wél significant meer views te genereren dan andere categorieën. De post-hocanalyse verduidelijkt dit verder.

## STAP 6: Post-hoc analyse

In [ ]:
# Dunn's test with Bonferroni correction
dunn_result = sp.posthoc_dunn(
    videos,
    val_col='views',
    group_col='sentiment_category',
    p_adjust='bonferroni'
)

# Show Dunn's test result
print("Dunn's test met Bonferroni-correctie:")
print(dunn_result)

### Resultaten en interpretatie post-hocanalyse

Om te bepalen tussen welke sentimentcategorieën het aantal views significant verschilt, werd een post-hocanalyse uitgevoerd met behulp van de **Dunn’s test**, inclusief **Bonferroni-correctie** voor multiple testing. Dit volgde op een significante Kruskal-Wallis toets (H = 15.98, *p* < 0.001).

De resultaten zijn als volgt:

| Vergelijking              | p-waarde (Bonferroni) |
|--------------------------|-----------------------|
| Positief vs Negatief     | 0.0004 ✅             |
| Positief vs Neutraal     | 0.2800 ❌             |
| Neutraal vs Negatief     | 0.2029 ❌             |

**Interpretatie**

De post-hocanalyse toont aan dat het **aantal views significant hoger ligt bij video's met overwegend positief sentiment** in vergelijking met video's met overwegend negatief sentiment (*p* = 0.0004). Er werd echter **geen significant verschil gevonden** tussen video’s met positief versus neutraal sentiment (*p* = 0.2800), noch tussen neutraal en negatief (*p* = 0.2029).

Deze bevindingen suggereren dat **extreem negatief sentiment in de comments geassocieerd is met een lager gemiddeld bereik**, terwijl neutraal en positief sentiment geen duidelijk onderscheid vertonen. Dit wijst mogelijk op een drempeleffect waarbij enkel uitgesproken negatief commentaar een negatieve invloed uitoefent op het kijkgedrag of de zichtbaarheid van de video.

De resultaten bieden gedeeltelijke ondersteuning voor hypothese 3: **een positief sentiment leidt niet per se tot meer views dan neutraal sentiment**, maar er is **wel een duidelijk verschil ten opzichte van negatief sentiment**. In praktijk betekent dit dat het vermijden van negatief sentiment in de commentaren mogelijk belangrijker is dan het genereren van uitgesproken positief sentiment.